In [5]:
import os

os.chdir('e:\\Programming\\R\\202204_project_community_tda')
# get the working directory
os.getcwd()

'e:\\Programming\\R\\202204_project_community_tda'

In [18]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

d:\Application\Conda\envs\geoneuro\lib\site-packages\geopandas\_compat.py:153: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()
C:\Users\Cover\AppData\Local\Temp\ipykernel_64492\642189860.py:3: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


In [273]:
file_path = './data/commune_nov_dataset/'
file_list = os.listdir(file_path)

In [183]:
time_buffer = pd.Timedelta('15 minutes')

df_list = [pd.read_csv(file_path + file_list[idx]) for idx in range(len(file_list))]

def convert_timestamp(df):
    df.pid = df.pid.astype('int')
    df.t_start, df.t_end = pd.to_datetime(df.t_start), pd.to_datetime(df.t_end)
    return df

df_list = [convert_timestamp(df) for df in df_list]


In [212]:
sum([len(df) for df in df_list])

1681797

In [19]:
# read the GIS shapefile data
gis_df = gpd.read_file('./data/communities/communes35.shp')
gis_df.name.unique()

In [184]:
# create 300m buffer for each community
def create_polygon_buffer(polygon_gdf, buffer_width_in_meters=250):
    meter_projection = 'EPSG:32650'
    if polygon_gdf.crs != meter_projection:
        polygon_gdf = polygon_gdf.to_crs(meter_projection)
    polygon_gdf['geometry'] = polygon_gdf.buffer(buffer_width_in_meters)
    polygon_gdf = polygon_gdf.to_crs('EPSG:4326')
    return polygon_gdf

In [185]:
# match the corresponding community
gis_df_list = [gis_df[gis_df.name.apply(lambda s: s in df_list[idx].commune[0])] for idx in range(len(df_list))]
gis_df_list = [create_polygon_buffer(gis_commune) for gis_commune in gis_df_list]

In [186]:
def residential_identify(df, gdf_commune):
    gdf_act = df.copy()
    gdf_act = gpd.GeoDataFrame(gdf_act, geometry=gpd.points_from_xy(gdf_act.longitude, gdf_act.latitude), crs='EPSG:4326')
    gdf_drop_within = gpd.sjoin(gdf_act, gdf_commune, how='inner', predicate='within')
    gdf_drop_within['commune_pts'] = True
    return gdf_drop_within

In [187]:
residential_identify_pair_list = list(zip(df_list, gis_df_list))
gdf_drop_within_list = [residential_identify(*pair) for pair in residential_identify_pair_list]

In [188]:
# kick the impossbile activities whose velocity exceed 120 km/h
clip_list = [create_polygon_buffer(gis_commune, 3e4) for gis_commune in gis_df_list]

In [190]:
def clip_impossible(df, gdf_clip):
    gdf_act = df.copy()
    gdf_act = gpd.GeoDataFrame(gdf_act, geometry=gpd.points_from_xy(gdf_act.longitude, gdf_act.latitude), crs='EPSG:4326')
    gdf_act = gpd.clip(gdf_act, gdf_clip)
    gdf_act.sort_values(by=['pid', 't_start'], inplace=True)
    return gdf_act

In [191]:
outlier_clip_pair_list = list(zip(df_list, clip_list))
gdf_possible_act_list = [clip_impossible(*pair) for pair in outlier_clip_pair_list]

In [192]:
gdf_possible_act, gdf_drop_within = pd.concat(gdf_possible_act_list), pd.concat(gdf_drop_within_list)

In [193]:
gdf_drop_within.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 561432 entries, 0 to 2934
Data columns (total 23 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   commune      561432 non-null  object        
 1   pid          561432 non-null  int32         
 2   t_start      561432 non-null  datetime64[ns]
 3   t_end        561432 non-null  datetime64[ns]
 4   longitude    561432 non-null  float64       
 5   latitude     561432 non-null  float64       
 6   X            561432 non-null  float64       
 7   Y            561432 non-null  float64       
 8   geometry     561432 non-null  geometry      
 9   index_right  561432 non-null  int64         
 10  id           561432 non-null  object        
 11  name         561432 non-null  object        
 12  address      561432 non-null  object        
 13  OBJECTID     561432 non-null  int64         
 14  Field1       561432 non-null  int64         
 15  Field2       561432 non-null  int

In [194]:
# join the drop within 
gdf_overlaied = pd.merge(gdf_possible_act, gdf_drop_within[['commune', 'pid', 't_start', 'commune_pts']], on=['commune', 'pid', 't_start'], how='left')
gdf_overlaied['commune_pts'] = gdf_overlaied['commune_pts'].fillna(False)
gdf_overlaied

,commune,pid,t_start,t_end,longitude,latitude,X,Y,geometry,commune_pts
0,益田村,861,2019-11-01 00:03:26,2019-11-03 09:51:18,114.048885,22.517184,196404.738631,2.493070e+06,POINT (114.04888 22.51718),True
1,益田村,861,2019-11-03 14:13:10,2019-11-03 16:07:56,114.048885,22.517184,196404.738631,2.493070e+06,POINT (114.04888 22.51718),True
2,益田村,861,2019-11-03 17:32:41,2019-11-03 18:46:08,114.065743,22.631277,198388.859223,2.505677e+06,POINT (114.06574 22.63128),False
3,益田村,861,2019-11-03 19:28:55,2019-11-03 22:11:32,113.915830,22.487217,182639.072479,2.490026e+06,POINT (113.91583 22.48722),False
4,益田村,861,2019-11-04 00:48:10,2019-11-04 12:59:41,114.048885,22.517184,196404.738631,2.493070e+06,POINT (114.04888 22.51718),True
...,...,...,...,...,...,...,...,...,...,...
1655965,龙光玖云著,37424317,2019-11-26 18:37:49,2019-11-26 20:58:06,114.337725,22.689845,226476.987565,2.511639e+06,POINT (114.33772 22.68984),False
1655966,龙光玖云著,37424317,2019-11-27 08:04:57,2019-11-29 21:34:52,114.353638,22.708351,228149.295700,2.513660e+06,POINT (114.35364 22.70835),False
1655967,龙光玖云著,37424317,2019-11-30 09:41:07,2019-11-30 10:15:57,114.324039,22.684798,225060.150495,2.511105e+06,POINT (114.32404 22.68480),True
1655968,龙光玖云著,37424317,2019-11-30 10:59:27,2019-11-30 15:22:47,114.379799,22.708169,230837.756819,2.513592e+06,POINT (114.37980 22.70817),False


In [196]:
gdf_overlaied.loc[gdf_overlaied.commune_pts, 'commune_upper_limit'] = gdf_overlaied.loc[gdf_overlaied.commune_pts, 't_end'] + time_buffer
gdf_overlaied.loc[gdf_overlaied.commune_pts, 'commune_lower_limit'] = gdf_overlaied.loc[gdf_overlaied.commune_pts, 't_start'] - time_buffer

In [197]:
gdf_overlaied.commune_upper_limit = gdf_overlaied.groupby(['commune', 'pid']).commune_upper_limit.ffill()
gdf_overlaied.commune_lower_limit = gdf_overlaied.groupby(['commune', 'pid']).commune_lower_limit.bfill()

In [208]:
gdf_filtered = gdf_overlaied.loc[(gdf_overlaied.commune_pts) | (~gdf_overlaied.commune_pts & (gdf_overlaied.t_start < gdf_overlaied.commune_upper_limit)) | (~gdf_overlaied.commune_pts & (gdf_overlaied.t_end > gdf_overlaied.commune_lower_limit))]

In [209]:
gdf_filtered = gdf_filtered.copy()
gdf_filtered.drop(columns = ['commune_upper_limit', 'commune_lower_limit'], inplace=True)

In [266]:
mapping_df = pd.read_csv('./data/commune_mapping.csv')
mapping_dict = {row.chinesename: row.filename for row in mapping_df.itertuples()}

In [261]:
res = pd.DataFrame(gdf_filtered.drop(columns=['geometry']))
res_grouped = res.groupby('commune')

In [262]:
commune_name = [grp for grp, _ in res_grouped]
commune_data = [df for _, df in res_grouped]

Taoyuan = '桃源村'
Taoyuan_indices = np.argwhere(np.array([name.find(Taoyuan) > -1 for name in commune_name])).flatten().tolist()

In [263]:
ans = [commune_data[idx] for idx in Taoyuan_indices]
ans = pd.concat(ans)

# remove the data list by index
commune_data = [df for i, df in enumerate(commune_data) if i not in Taoyuan_indices]
commune_data.insert(Taoyuan_indices[0], ans)

In [267]:
commune_data.sort(key=lambda df: mapping_dict[df.commune.iloc[0]])

In [275]:
# export the data
for i, df in enumerate(commune_data):
    df.to_csv(f'./data/quinze_juin_dataset/{file_list[i]}', index=False)

Sketch